<div style="text-align: center;">
    <img src="https://sm.ign.com/ign_in/screenshot/default/xbox-gamescom-2023_nqc8.jpg" alt="Xbox Gamescom 2023" style="width: 60%; height: auto;">
</div>

# Xbox Game Pass - Portfolio Data Analysis

# Introduction
Xbox Game Pass is a subscription service that provides access to a library of hundreds of games across multiple genres. In this project, I will explore the Xbox Game Pass portfolio based on the dataset’s last update in 2022. I will also seek to enhance the dataset by adding game genres, providing a more in-depth analysis.

# Goals
1. Enhance the dataset by adding game genres, allowing for more comprehensive analysis.
2. Perform exploratory data analysis to uncover insights into game genres and the performance of the Xbox Game Pass portfolio.
3. Visualise the data to ensure results are clear, accessible, and easy to interpret.

In [ ]:
import numpy as np 
import pandas as pd 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Imports necessary libraries for data manipulation and visualisation

# 1. Game Genres

1. Expanding the dataset to include game genres was achieved by uploading the original dataset (CSV file) into an LLM (Grok) and prompting it to research the genre for each game and then to create a new column for all 400+ games.
2. Grok was able to research and perform this task by assigning each game in the list the correct genre, including sub-genres.
3. Some data cleaning was performed in Google Sheets by separating primary genres from sub-genres, preparing the dataset for analysis.

In [ ]:
# New enhanced CSV with seperated genres
xbox = pd.read_csv('/kaggle/input/enhanced-xbox-game-pass-genres/Enhanced_Gamepass_Games_v2.csv')

xbox.head()

# 2. Data Analysis & Visualisations

## Ratings by Genre

**What are the Average Ratings by Genres? (Main Genre)**

In [ ]:
# Group by Main_Genre and calculate avg rating and count of games
main_genre_stats = xbox.groupby('Main_Genre').agg({
    'RATING': 'mean',
    'GAME': 'count'
}).rename(columns={'RATING': 'Average_Rating', 'GAME': 'Game_Count'}).reset_index()

# Sort by Average_Rating in descending order
main_genre_stats = main_genre_stats.sort_values('Average_Rating', ascending=False)

main_genre_stats['Average_Rating'] = main_genre_stats['Average_Rating'].round(2)

main_genre_stats.head(25)

Although genres such as **Metroidvania** appear to have the highest rating, this is not a reliable indicator since it is based on only one game, compared with **Action**, which includes 182 games

Let's combine the sub-genre's and count them to get a better picture.

In [ ]:
# Clean columns (strip whitespace to fix dupelicates)
for col in ['Main_Genre', 'second_genre', 'third_genre', 'fourth_genre']:
    xbox[col] = xbox[col].str.strip()

# Combine the genre columns into a single column
genres_combined = pd.melt(
    xbox,
    id_vars=['GAME', 'RATING', 'GAMERS'],
    value_vars=['Main_Genre', 'second_genre', 'third_genre', 'fourth_genre'],
    value_name='Genre'
).dropna(subset=['Genre'])

# Group by Genre and calculate average rating and count
all_genre_stats = genres_combined.groupby('Genre').agg({
    'RATING': 'mean',
    'GAME': 'count'
}).rename(columns={'RATING': 'Average_Rating', 'GAME': 'Game_Count'}).reset_index()

all_genre_stats['Average_Rating'] = all_genre_stats['Average_Rating'].round(2)
all_genre_stats = all_genre_stats.sort_values('Game_Count', ascending=False)

all_genre_stats.head(25)

Let's visualise these results:

In [ ]:
# Create an interactive bar plot for top 15 genres by Game_Count using Plotly
fig = px.bar(all_genre_stats.head(15), 
             x='Average_Rating', 
             y='Genre', 
             color='Game_Count',  # Use Game_Count for color variation
             color_continuous_scale='Viridis',
             title='Average Rating by Genre',
             labels={'Average_Rating': 'Average Rating (1-5)', 'Genre': 'Genre', 'Game_Count': 'Game Count'},
             category_orders={'Genre': all_genre_stats.head(15).sort_values('Average_Rating', ascending=False)['Genre'].tolist()},
             height=600,  # Adjust height
             width=800)   # Adjust width

# Update layout
fig.update_layout(
    xaxis_title="Average Rating (1-5)",
    yaxis_title="Genre",
    coloraxis_colorbar_title="Game Count",
    showlegend=True
)

# Show the interactive plot
fig.show(renderer='iframe')

Genres **Action** and **Adventure** have a high number of games (game counts) and a high average rating, suggesting these are the most played and most popular genres alongside **RPG**.

## Ratings vs Completion Rate

**Do genres with higher completion percentage (COMP %) have better ratings, indicating player satisfaction?** 

I'll focus on the games primary or "main genre" for this analysis (as opposed to including sub-genres).

In [ ]:
# Group by Main_Genre and calculate mean COMP % and RATING
genre_comp = xbox.groupby('Main_Genre').agg({
    'COMP %': 'mean',
    'RATING': 'mean',
    'GAME': 'count'  # Count of games per genre for context
}).rename(columns={'GAME': 'Game_Count'}).reset_index()

# Sort by COMP % to identify genres with high completion rates
genre_comp = genre_comp.sort_values('COMP %', ascending=False)

genre_comp['COMP %'] = genre_comp['COMP %'].round(2)
genre_comp['RATING'] = genre_comp['RATING'].round(2)

genre_comp.head(10)

**Adventure** and **Arcade** have the highest completion rates, but how does this look when we factor in game ratings?

In [ ]:
# Interactive scatter plot with hover
fig = px.scatter(genre_comp, x='COMP %', y='RATING', color='Main_Genre',
                 title='Average Completion Rate vs. Rating by Genre',
                 labels={'COMP %': 'Average Completion Rate (%)', 'RATING': 'Average Rating (out of 5)'},
                 hover_data=['Main_Genre'])  # Hover shows genre

fig.update_layout(
    width=800,
    height=600,
    showlegend=True
)
fig.show(renderer='iframe')

We can see a few outliers:

1. Arcade
2. Adventure
3. Platformer
4. Simulation
5. Metroidvania

Do games with higher completion rates have better ratings? 

On average, games with a higher completion percentage % **do not** indicate higher ratings, with the exception of the genre **Platformer**, which has the third highest completion rate *and* the third highest rating.


Finally, let's analyse:

## Player Distribution Across Genres

**Which genres have the highest player engagement?** Let's highlight the most popular genres on Game Pass, potentially guiding content strategy.

> Note on Gamer Counts: Multi-genre games (e.g., Action-Adventure) add their full player count to each genre, leading to double-counting. This shows genre exposure, not unique players per genre.

In [ ]:
# Ensure GAMERS is numeric in the original DataFrame before melting
xbox['GAMERS'] = pd.to_numeric(xbox['GAMERS'].str.replace(',', ''), errors='coerce')

# Recreate genres_combined with proper handling to avoid GAMERS duplication
genres_combined_gamers = pd.melt(
    xbox,
    id_vars=['GAME', 'GAMERS'],  # Focus on GAME and GAMERS
    value_vars=['Main_Genre', 'second_genre', 'third_genre', 'fourth_genre'],
    value_name='Genre'
).dropna(subset=['Genre'])

# Group by Genre and calculate total gamers and unique game count
genre_gamers_dist = genres_combined_gamers.groupby('Genre').agg({
    'GAMERS': 'sum',  # Sum the numeric GAMERS values
    'GAME': 'nunique'  # Count unique games to avoid overcounting
}).rename(columns={'GAMERS': 'Total_Gamers', 'GAME': 'Game_Count'}).reset_index()

# Ensure Total_Gamers is numeric and handle any remaining issues
genre_gamers_dist['Total_Gamers'] = pd.to_numeric(genre_gamers_dist['Total_Gamers'], errors='coerce')

# Round Total_Gamers to the nearest million for readability
genre_gamers_dist['Total_Gamers_Millions'] = (genre_gamers_dist['Total_Gamers'] / 1_000_000).round(0) * 1_000_000

# Sort by Total_Gamers in descending order
genre_gamers_dist = genre_gamers_dist.sort_values('Total_Gamers', ascending=False)

# Display the top 15 genres with formatted values
genre_gamers_dist.head(15)

In [ ]:
# Create an interactive bar plot for Player Distribution Across Genres
fig = px.bar(genre_gamers_dist.head(10), 
             x='Total_Gamers',
             y='Genre',
             color='Game_Count',  # Use Game_Count for color variation
             color_continuous_scale='Viridis',
             title='Player Exposure Across Genres (Top 10 by Total Gamers)',
             labels={'Total_Gamers': 'Total Gamers (Thousands)', 'Genre': 'Genre', 'Game_Count': 'Game Count'},
             height=600,
             width=800)

fig.update_layout(
    xaxis_title="Total Gamers",
    yaxis_title="Genre",
    coloraxis_colorbar_title="Game Count",
    showlegend=True
)

fig.show(renderer='iframe')

Genres **Action** & **Adventure** have the highest player exposure. A higher game count strongly correlates to a higher player distribution.

# Summary


> 1. Genres **Action** and **Adventure** have a high game count and a high average rating, suggesting these are the most popular played and highest reached genres across Xbox Game Pass.
> 2. Games with a higher completion percentage % **do not** indicate higher ratings, with the exception of **Platformer** games.
> 3. Games tagged as **Action** and **Adventure** equate to around 50% of games on Game Pass by total gamers (40M).


## Reflection

For future analysis of this dataset, I would like to understand:

1. Are popular game genres rated higher due to broader appeal? or do niche games (lower player count) receive higher ratings from smaller more dedicated fans?
2. Is there a correlation between time to complete (hours) and higher or lower ratings?
3. I would like to refactor the code by implementing a combined melt to reduce code duplication.